# Travel Agency RAG System - Evaluation

This notebook demonstrates the recall improvements achieved by the improved retrieval system compared to the baseline approach.

## Key Improvements:
1. **Structured Field Extraction**: LLM extracts activities/services during indexing
2. **Query Rewriting**: LLM converts natural language queries into structured filters
3. **Structured Filtering**: BM25 + structured field matching for high precision


In [ ]:
import sys
import os

# Add parent directory to path
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('')))
sys.path.append(parent_dir)

from retrieval.baseline_retriever import BaselineRetriever
from retrieval.improved_retriever import ImprovedRetriever
import json


## Test Queries

These queries are designed to test the system's ability to handle:
- Location-specific queries
- Activity-based queries
- Complex multi-criteria queries


In [ ]:
test_queries = [
    "Snorkeling near Lisbon",
    "Wine tasting in Tuscany",
    "City tours in Paris",
    "Hiking in Iceland",
    "Culinary tours in Tokyo",
    "Beaches and snorkeling",
    "Photography tours",
    "Historical tours in Japan"
]


## Initialize Retrievers


In [ ]:
index_dir = os.path.join(parent_dir, "indexes")
baseline_path = os.path.join(index_dir, "baseline")
improved_path = os.path.join(index_dir, "improved")

baseline_retriever = BaselineRetriever(baseline_path)
improved_retriever = ImprovedRetriever(improved_path)


## Compare Results for Each Query


In [ ]:
for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}")
    
    # Baseline
    baseline_results = baseline_retriever.search(query, limit=5)
    print(f"\n📊 BASELINE: {len(baseline_results)} results")
    for i, r in enumerate(baseline_results[:3], 1):
        print(f"  {i}. {r['name']} ({r['doc_type']}) - Score: {r['score']:.3f}")
    
    # Improved
    improved_result = improved_retriever.search(query, limit=5)
    print(f"\n✨ IMPROVED: {improved_result['num_results']} results")
    if improved_result['rewritten_query']:
        rq = improved_result['rewritten_query']
        print(f"  Rewritten: City={rq.get('city')}, Activities={rq.get('activities')}")
    for i, r in enumerate(improved_result['results'][:3], 1):
        print(f"  {i}. {r['name']} ({r['doc_type']}) - Score: {r['score']:.3f}")
        if r.get('activities'):
            print(f"     Activities: {', '.join(r['activities'][:3])}")


## Run Full Evaluation

Run the evaluation script for detailed metrics:


In [ ]:
!python evaluation/evaluate_recall.py


## Key Findings

The improved system demonstrates:
1. **Higher Recall**: Structured filtering ensures relevant documents aren't missed
2. **Better Precision**: Query rewriting extracts exact intent
3. **Handles Variations**: "snorkeling" matches even if document says "snorkel"
4. **Location-Aware**: Correctly filters by city/country when specified
